# Cloud and sea tile classifier

It has been identified that ESRGAN in both pretrain PSNR mode and GAN mode struggles with super-resoluting satellite image tiles completely covered by either sea surface or opaque clouds. In addition, both areas of interest, the harbors of Toulon and La Spezia, has lots of sea surface (approaching 50%). Left without intervention around 50% of tiles will only consist of sea and opaque clouds. It is assumed that this leads to an unwanted imbalance in what we want the model to be optimized to perform on.

There are several ways to mitigate this imbalance. One way would be to manually draw a sea surface polygon in a GIS software and undersample tiles extracted from within this polygon. A downside of this approach is that interesting features (ships) within the sea surface polygon would also be undersampled.

Another approach would be to train a cloud and sea tile classifier to detect the the unwanted tiles and discard all or a significant proportion of these tiles before training. This approach has the benefit of addressing the problem head on. The main downside of the approach is that it might be time-consuming to label tiles. However it is hypothesized that relatively little training data is needed to train a modern neural net classifier on such a *simple* classification task.

In [1]:
import pickle
import geopandas
import pandas as pd
import pathlib
import datetime
import rasterio
import rasterio.plot
import tensorflow as tf

from modules.tile_generator import *
from modules.helpers import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.cloudsea_classifier import *

# Check GPUs:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
tf.sysconfig.get_build_info()

1 Physical GPUs, 1 Logical GPUs


{'cuda_version': '64_101',
 'cudnn_version': '64_7',
 'cuda_compute_capabilities': ['sm_35',
  'sm_37',
  'sm_52',
  'sm_60',
  'sm_61',
  'compute_70'],
 'cpu_compiler': 'C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.26.28801/bin/HostX64/x64/cl.exe',
 'is_rocm_build': False,
 'is_cuda_build': True,
 'msvcp_dll_names': 'msvcp140.dll,msvcp140_1.dll',
 'nvcuda_dll_name': 'nvcuda.dll',
 'cudart_dll_name': 'cudart64_101.dll',
 'cudnn_dll_name': 'cudnn64_7.dll'}

In [2]:
# Toggles whether to actually do the generation on this run
# Be careful with setting these to True if tiles are already labelled!
# New tiles will overwrite old tiles and labels are trash
GENERATE_NEW_TILES = False
CONVERT_TO_PNG = False
CREATE_LABEL_CSV = False

with open('metadata_df.pickle', 'rb') as file:
    meta = pickle.load(file)
# Path to location where individual satellite images are located
DATA_PATH = 'data/toulon-laspezia/'
DATA_PATH_TILES = 'data/toulon-laspezia-cloud-sea-classifier/'

SENSORS = ['WV02', 'GE01', 'WV03_VNIR']
AREAS = ['La_Spezia', 'Toulon']
meta = meta.loc[meta['sensorVehicle'].isin(SENSORS)]
meta = meta.loc[meta['area_name'].isin(AREAS)]

N_IMAGES = len(meta.index)

#96x96, 128x128, 196x196, 384x384 -- All tiles are squares
TILE_SIZES = [96, 128, 196, 384]
# number of tiles to generate at each tile size
N_TILES = {96: 500, 128: 1000, 196: 500, 384: 500}
N_TILES_TOTAL = sum(N_TILES.values())

print(N_IMAGES)
print(N_TILES)
print(N_TILES_TOTAL)

TRAIN_TILE_SIZE = 224

PAN_OR_MS_OR_BOTH = 'pan'
if PAN_OR_MS_OR_BOTH == 'pan':
    TRAIN_TILE_BANDS = 1
elif PAN_OR_MS_OR_BOTH == 'ms':
    TRAIN_TILE_BANDS = 4
elif PAN_OR_MS_OR_BOTH == 'both':
    TRAIN_TILE_BANDS = 5

RESIZE_METHOD = 'bilinear'

BATCH_SIZE = 16

VAL_SPLIT = 0.3

73
{96: 500, 128: 1000, 196: 500, 384: 500}
2500


## Allocate n_tiles to every image (weighted by size of image)

In [3]:
if GENERATE_NEW_TILES:
    meta = allocate_tiles(meta, by_partition=False, n_tiles_total=N_TILES[96], new_column_name='n_tiles_96')
    meta = allocate_tiles(meta, by_partition=False, n_tiles_total=N_TILES[128], new_column_name='n_tiles_128')
    meta = allocate_tiles(meta, by_partition=False, n_tiles_total=N_TILES[196], new_column_name='n_tiles_196')
    meta = allocate_tiles(meta, by_partition=False, n_tiles_total=N_TILES[384], new_column_name='n_tiles_384')
    meta

## Generate tiles to disk

In [4]:
if GENERATE_NEW_TILES:
    meta['n_tiles'] = 0
    for tile_size in TILE_SIZES:
        pathlib.Path(DATA_PATH_TILES).joinpath(str(tile_size)).mkdir()
        tile_size_ms = int(tile_size/4)
        meta['n_tiles'] = meta[str('n_tiles_'+str(tile_size))]
        generate_all_tiles(meta, save_dir = str(DATA_PATH_TILES+'/'+str(tile_size)), 
                           ms_height_width=(tile_size_ms,tile_size_ms), sr_factor=4, 
                           cloud_sea_removal=False)

## Flatten the directory structure after generation

Lots of foor loops in order to do one change at a time.

In [5]:
if GENERATE_NEW_TILES:
    # Remove train/val/test directories
    for tilesize_dir in pathlib.Path(DATA_PATH_TILES).iterdir():
        for partition_dir in tilesize_dir.iterdir():
            for image_dir in partition_dir.iterdir():
                dest = tilesize_dir.joinpath(image_dir.stem)
                source = image_dir
                source.rename(dest)
            partition_dir.rmdir()

    # Add tile size to filenames
    for tilesize_dir in pathlib.Path(DATA_PATH_TILES).iterdir():
        for image_dir in tilesize_dir.iterdir():
            for ms_pan_dir in image_dir.iterdir():
                for tile in ms_pan_dir.iterdir():
                    new_tile_name = str(tilesize_dir.stem+'-'+tile.name)
                    new_path = ms_pan_dir.joinpath(new_tile_name)
                    tile.rename(new_path)

    # Completely flatten file structure, remove tile size directories
    for tilesize_dir in pathlib.Path(DATA_PATH_TILES).iterdir():
        for image_dir in tilesize_dir.iterdir():
            for ms_pan_dir in image_dir.iterdir():
                for tile in ms_pan_dir.iterdir():
                    new_dir = pathlib.Path(DATA_PATH_TILES).joinpath(image_dir.stem, ms_pan_dir.name)
                    new_dir.mkdir(parents=True, exist_ok=True)
                    new_path = new_dir.joinpath(tile.name)
                    tile.rename(new_path)
                ms_pan_dir.rmdir()
            image_dir.rmdir()
        tilesize_dir.rmdir()

    # Add image_int_uid to filenames and flatten structure completely
    for image_dir in pathlib.Path(DATA_PATH_TILES).iterdir():
        if image_dir.stem == 'ms' or image_dir.stem == 'pan':
            continue
        for ms_pan_dir in image_dir.iterdir():
            for tile in ms_pan_dir.iterdir():
                int_uid = get_int_uid(meta, image_dir.stem)
                new_tile_name = str(str(int_uid).zfill(2)+'-'+tile.name)
                new_dir = pathlib.Path(DATA_PATH_TILES).joinpath(ms_pan_dir.stem)
                new_dir.mkdir(parents=True, exist_ok=True)
                new_path = new_dir.joinpath(new_tile_name)
                tile.rename(new_path)
            ms_pan_dir.rmdir()
        image_dir.rmdir()

# List all tif files
tif_paths = [file for file in pathlib.Path(DATA_PATH_TILES).glob('**/*.tif')]
tif_paths_ms = tif_paths[:2500]
tif_paths_pan = tif_paths[2500:]

# Divide by 2 because each tile consists of 1 MS + 1 PAN
print('Number of tiles generated and present in flat file structure:', str(int(len(tif_paths)/2)))

Number of tiles generated and present in flat file structure: 2500


# Convert to png
While the input to the actual cloud/sea classifier is tif files it is practical to also convert the image tiles to png. This makes labelling easier.

In [6]:
if CONVERT_TO_PNG:
    for tif_path in tif_paths:
        ms_or_pan = tif_path.parent.stem
        
        # sensor type is needed for conversion of ms to rgb png::
        int_uid = int(tif_path.stem[:2])
        string_uid = get_string_uid(meta, int_uid)
        sensor = get_sensor(meta, string_uid)
        
        # saves png to disk
        geotiff_to_png(tif_path, ms_or_pan=ms_or_pan, scale=True, stretch_img=True, sensor=sensor)

# List all png files
png_paths = [file for file in pathlib.Path(DATA_PATH_TILES).glob('**/*.png')]

# Divide by 2 because each tile consists of 1 MS + 1 PAN
print('Number of tiles generated and present in flat file structure:', str(int(len(png_paths)/2)))

Number of tiles generated and present in flat file structure: 2500


# Create label csv file
Labels are `None` before manual labelling

In [7]:
if CREATE_LABEL_CSV:
    label_df = pd.DataFrame([tif_path.stem for tif_path in tif_paths[:N_TILES_TOTAL]], columns=['tile_uid'])
    label_df['cloud-sea'] = None
    label_df.to_csv(pathlib.Path(DATA_PATH_TILES).joinpath('labels-to-be.csv'), index=False)

# Labelling
*... 4 tedious labelling hours later...*

# Loading the labeled csv

Our `load_and_populate_label_df` function also extracts `sensor`, `tile_size` and `img_uid` from the `tile_uid` column. This is used later when preparing data for training.

In [8]:
label_df = load_and_populate_label_df(DATA_PATH_TILES + '/labels.csv', meta)
label_df

,tile_uid,cloud-sea,sensor,tile_size,img_uid
0,00-128-00000,1,GE01,128,GE01_La_Spezia_2009_09_25_011651186010_0
1,00-128-00001,1,GE01,128,GE01_La_Spezia_2009_09_25_011651186010_0
2,00-128-00002,1,GE01,128,GE01_La_Spezia_2009_09_25_011651186010_0
3,00-128-00003,0,GE01,128,GE01_La_Spezia_2009_09_25_011651186010_0
4,00-128-00004,0,GE01,128,GE01_La_Spezia_2009_09_25_011651186010_0
...,...,...,...,...,...
2495,72-96-00007,1,WV03_VNIR,96,WV03_VNIR_Toulon_2019_02_11_011650889010_0
2496,72-96-00008,1,WV03_VNIR,96,WV03_VNIR_Toulon_2019_02_11_011650889010_0
2497,72-96-00009,0,WV03_VNIR,96,WV03_VNIR_Toulon_2019_02_11_011650889010_0
2498,72-96-00010,1,WV03_VNIR,96,WV03_VNIR_Toulon_2019_02_11_011650889010_0


## Preprocessing of images and labels

Images in have different resolution. Classifier model architecture `EfficientNet` requires fixed image sizes as input so images are resized to `TRAIN_TILE_SIZE_PAN = 224`. [Native EfficientNet image sizes](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

Note that `EfficientNet` has rescaling and normalizing layers as their first layers so such preprocessing is not required in advance. In fact it seems like such preprocessing hurts performance of the model. We can therefore keep image tiles with `dtype=uint16`.

In [9]:
X, y = prepare_for_training(label_df, tif_paths_pan, tif_paths_ms,
                            pan_or_ms_or_both=PAN_OR_MS_OR_BOTH,
                            pan_tile_size=TRAIN_TILE_SIZE, ms_tile_size=TRAIN_TILE_SIZE, 
                            resize_method=RESIZE_METHOD)

Verification OK. All 2500 image tile UIDs match.
Loaded 0 images
Loaded 100 images
Loaded 200 images
Loaded 300 images
Loaded 400 images
Loaded 500 images
Loaded 600 images
Loaded 700 images
Loaded 800 images
Loaded 900 images
Loaded 1000 images
Loaded 1100 images
Loaded 1200 images
Loaded 1300 images
Loaded 1400 images
Loaded 1500 images
Loaded 1600 images
Loaded 1700 images
Loaded 1800 images
Loaded 1900 images
Loaded 2000 images
Loaded 2100 images
Loaded 2200 images
Loaded 2300 images
Loaded 2400 images
Finished preparing 2500 images and labels for training!
X.shape: (2500, 224, 224, 1)
y.shape: (2500,)


## Model

In [10]:
model = build_model(augment=True, input_shape=(TRAIN_TILE_SIZE, TRAIN_TILE_SIZE, TRAIN_TILE_BANDS))
model.summary()

Model: "EfficientNetB0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
img_augmentation (Sequential)   (None, 224, 224, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 1)  0           img_augmentation[0][0]           
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 1)  3           rescaling[0][0]                  
_____________________________________________________________________________________

In [11]:
pretrain_model_name = str('cloud-sea-classifier-effnetb0-pan-augm')
log_dir = pathlib.Path(str('logs/cloud-sea-classifier/fit/' + pretrain_model_name 
                           + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                      histogram_freq=10, 
                                                      write_graph=False, 
                                                      write_images=False,
                                                      update_freq='epoch')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = str('models/cloud-sea-classifier/' + pretrain_model_name + '-{epoch:02d}-{val_loss:.6f}.h5'), 
    monitor = "val_acc",
    save_best_only = False,
    save_weights_only = True,
    )

In [ ]:
EPOCHS = 100
model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS,
          validation_split=VAL_SPLIT, initial_epoch=0,
          callbacks=[checkpoint_callback, tensorboard_callback])

# Hyperparameter tuning

In [12]:
RESIZE_METHODS = ['bilinear', 'nearest', 'bicubic']
PAN_MS_BOTH = ['pan', 'ms', 'both']
LEARNING_RATES = [0.001, 0.0005, 0.0001]

EPOCHS = 200

for resize_method in RESIZE_METHODS:
    for pan_ms_both in PAN_MS_BOTH:
        if pan_ms_both == 'pan':
            train_tile_bands = 1
        elif pan_ms_both == 'ms':
            train_tile_bands = 4
        elif pan_ms_both == 'both':
            train_tile_bands = 5
        for learning_rate in LEARNING_RATES:
                
            X, y = prepare_for_training(label_df, tif_paths_pan, tif_paths_ms,
                                        pan_or_ms_or_both=pan_ms_both,
                                        pan_tile_size=TRAIN_TILE_SIZE, ms_tile_size=TRAIN_TILE_SIZE, 
                                        resize_method=resize_method)

            model = build_model(augment=True, 
                                input_shape=(TRAIN_TILE_SIZE, TRAIN_TILE_SIZE, train_tile_bands), 
                                learning_rate=learning_rate)

            pretrain_model_name = str('cloudsea-effb0-augm-' + resize_method 
                                      + '-' + pan_ms_both + '-' + str(learning_rate) + '-')
            log_dir = pathlib.Path(str('logs/cloud-sea-classifier/fit/' + pretrain_model_name 
                                       + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')))
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                                  histogram_freq=10, 
                                                                  write_graph=False, 
                                                                  write_images=False,
                                                                  update_freq='epoch')

            checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath = str('models/cloud-sea-classifier/' + pretrain_model_name + '-{epoch:02d}-{val_loss:.6f}.h5'), 
                monitor = "val_acc",
                save_best_only = False,
                save_weights_only = True,
                )

            model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS,
                      validation_split=VAL_SPLIT, initial_epoch=0,
                      callbacks=[checkpoint_callback, tensorboard_callback])

Verification OK. All 2500 image tile UIDs match.
Loaded 0 images
Loaded 100 images
Loaded 200 images
Loaded 300 images
Loaded 400 images
Loaded 500 images
Loaded 600 images
Loaded 700 images
Loaded 800 images
Loaded 900 images
Loaded 1000 images
Loaded 1100 images
Loaded 1200 images
Loaded 1300 images
Loaded 1400 images
Loaded 1500 images
Loaded 1600 images
Loaded 1700 images
Loaded 1800 images
Loaded 1900 images
Loaded 2000 images
Loaded 2100 images
Loaded 2200 images
Loaded 2300 images
Loaded 2400 images
Finished preparing 2500 images and labels for training!
X.shape: (2500, 224, 224, 1)
y.shape: (2500,)
Epoch 1/200
  1/110 [..............................] - ETA: 0s - loss: 1.8317 - accuracy: 0.3750WARNING:tensorflow:From c:\users\ono008\master\multispectral-super-resolution\env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.exp

110/110 [==============================] - 16s 147ms/step - loss: 0.1635 - accuracy: 0.9497 - val_loss: 0.3280 - val_accuracy: 0.9173
Epoch 49/200
110/110 [==============================] - 16s 146ms/step - loss: 0.1851 - accuracy: 0.9411 - val_loss: 2.3358 - val_accuracy: 0.8787
Epoch 50/200
110/110 [==============================] - 16s 147ms/step - loss: 0.1547 - accuracy: 0.9526 - val_loss: 0.2160 - val_accuracy: 0.9333
Epoch 51/200
110/110 [==============================] - 16s 150ms/step - loss: 0.1970 - accuracy: 0.9480 - val_loss: 0.2051 - val_accuracy: 0.9307
Epoch 52/200
110/110 [==============================] - 16s 149ms/step - loss: 0.1583 - accuracy: 0.9509 - val_loss: 0.2442 - val_accuracy: 0.9253
Epoch 53/200
110/110 [==============================] - 16s 147ms/step - loss: 0.1457 - accuracy: 0.9520 - val_loss: 0.1913 - val_accuracy: 0.9427
Epoch 54/200
110/110 [==============================] - 16s 148ms/step - loss: 0.1585 - accuracy: 0.9491 - val_loss: 0.1936 - val_a

Epoch 104/200
110/110 [==============================] - 16s 148ms/step - loss: 0.1037 - accuracy: 0.9697 - val_loss: 0.1750 - val_accuracy: 0.9453
Epoch 105/200
110/110 [==============================] - 16s 148ms/step - loss: 0.0858 - accuracy: 0.9737 - val_loss: 0.2044 - val_accuracy: 0.9373
Epoch 106/200
110/110 [==============================] - 16s 147ms/step - loss: 0.0978 - accuracy: 0.9749 - val_loss: 0.1700 - val_accuracy: 0.9507
Epoch 107/200
110/110 [==============================] - 16s 147ms/step - loss: 0.1052 - accuracy: 0.9674 - val_loss: 0.1739 - val_accuracy: 0.9520
Epoch 108/200
110/110 [==============================] - 16s 148ms/step - loss: 0.0979 - accuracy: 0.9731 - val_loss: 0.1835 - val_accuracy: 0.9413
Epoch 109/200
110/110 [==============================] - 16s 147ms/step - loss: 0.1121 - accuracy: 0.9709 - val_loss: 0.1939 - val_accuracy: 0.9427
Epoch 110/200
110/110 [==============================] - 16s 148ms/step - loss: 0.0977 - accuracy: 0.9680 - val_

110/110 [==============================] - 16s 147ms/step - loss: 0.0869 - accuracy: 0.9777 - val_loss: 0.1347 - val_accuracy: 0.9560
Epoch 160/200
110/110 [==============================] - 16s 147ms/step - loss: 0.0764 - accuracy: 0.9806 - val_loss: 0.1641 - val_accuracy: 0.9587
Epoch 161/200
110/110 [==============================] - 17s 151ms/step - loss: 0.0857 - accuracy: 0.9777 - val_loss: 0.1449 - val_accuracy: 0.9533
Epoch 162/200
110/110 [==============================] - 16s 148ms/step - loss: 0.0726 - accuracy: 0.9771 - val_loss: 0.1442 - val_accuracy: 0.9493
Epoch 163/200
110/110 [==============================] - 16s 147ms/step - loss: 0.0994 - accuracy: 0.9766 - val_loss: 0.1393 - val_accuracy: 0.9600
Epoch 164/200
110/110 [==============================] - 16s 146ms/step - loss: 0.0923 - accuracy: 0.9737 - val_loss: 0.2022 - val_accuracy: 0.9293
Epoch 165/200
110/110 [==============================] - 16s 148ms/step - loss: 0.1139 - accuracy: 0.9634 - val_loss: 0.1446 -

110/110 [==============================] - 16s 146ms/step - loss: 0.3639 - accuracy: 0.8549 - val_loss: 0.5696 - val_accuracy: 0.7973
Epoch 9/200
110/110 [==============================] - 16s 146ms/step - loss: 0.3703 - accuracy: 0.8434 - val_loss: 0.4288 - val_accuracy: 0.8280
Epoch 10/200
110/110 [==============================] - 16s 147ms/step - loss: 0.3164 - accuracy: 0.8817 - val_loss: 0.5379 - val_accuracy: 0.8000
Epoch 11/200
110/110 [==============================] - 17s 151ms/step - loss: 0.2859 - accuracy: 0.8874 - val_loss: 0.4682 - val_accuracy: 0.8240
Epoch 12/200
110/110 [==============================] - 16s 146ms/step - loss: 0.2809 - accuracy: 0.8914 - val_loss: 0.5330 - val_accuracy: 0.7587
Epoch 13/200
110/110 [==============================] - 16s 146ms/step - loss: 0.2623 - accuracy: 0.9051 - val_loss: 0.2703 - val_accuracy: 0.8920
Epoch 14/200
110/110 [==============================] - 16s 146ms/step - loss: 0.4200 - accuracy: 0.8989 - val_loss: 0.3194 - val_ac

...

110/110 [==============================] - 18s 165ms/step - loss: 0.0671 - accuracy: 0.9806 - val_loss: 0.8108 - val_accuracy: 0.7813
Epoch 168/200
110/110 [==============================] - 18s 164ms/step - loss: 0.0629 - accuracy: 0.9817 - val_loss: 0.2096 - val_accuracy: 0.9227
Epoch 169/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0724 - accuracy: 0.9800 - val_loss: 0.8728 - val_accuracy: 0.7600
Epoch 170/200
110/110 [==============================] - 18s 166ms/step - loss: 0.0662 - accuracy: 0.9800 - val_loss: 0.6646 - val_accuracy: 0.7893
Epoch 171/200
110/110 [==============================] - 19s 169ms/step - loss: 0.0723 - accuracy: 0.9789 - val_loss: 0.5236 - val_accuracy: 0.8093
Epoch 172/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0590 - accuracy: 0.9840 - val_loss: 0.6658 - val_accuracy: 0.8213
Epoch 173/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0669 - accuracy: 0.9800 - val_loss: 0.8848 -

Epoch 17/200
110/110 [==============================] - 18s 165ms/step - loss: 0.3179 - accuracy: 0.8697 - val_loss: 0.5945 - val_accuracy: 0.6760
Epoch 18/200
110/110 [==============================] - 18s 166ms/step - loss: 0.3262 - accuracy: 0.8720 - val_loss: 0.6847 - val_accuracy: 0.6853
Epoch 19/200
110/110 [==============================] - 18s 165ms/step - loss: 0.3451 - accuracy: 0.8709 - val_loss: 0.5703 - val_accuracy: 0.7093
Epoch 20/200
110/110 [==============================] - 18s 164ms/step - loss: 0.3584 - accuracy: 0.8794 - val_loss: 0.8147 - val_accuracy: 0.6947
Epoch 21/200
110/110 [==============================] - 18s 168ms/step - loss: 0.3330 - accuracy: 0.8766 - val_loss: 0.5443 - val_accuracy: 0.7613
Epoch 22/200
110/110 [==============================] - 18s 164ms/step - loss: 0.3113 - accuracy: 0.8823 - val_loss: 0.7015 - val_accuracy: 0.7613
Epoch 23/200
110/110 [==============================] - 18s 165ms/step - loss: 0.3462 - accuracy: 0.8640 - val_loss: 0

Epoch 73/200
110/110 [==============================] - 18s 165ms/step - loss: 0.1371 - accuracy: 0.9554 - val_loss: 0.4467 - val_accuracy: 0.8173
Epoch 74/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1314 - accuracy: 0.9549 - val_loss: 0.5918 - val_accuracy: 0.8227
Epoch 75/200
110/110 [==============================] - 18s 165ms/step - loss: 0.1554 - accuracy: 0.9583 - val_loss: 0.6031 - val_accuracy: 0.8120
Epoch 76/200
110/110 [==============================] - 18s 166ms/step - loss: 0.1759 - accuracy: 0.9560 - val_loss: 0.4943 - val_accuracy: 0.8347
Epoch 77/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1613 - accuracy: 0.9560 - val_loss: 0.7324 - val_accuracy: 0.7960
Epoch 78/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1510 - accuracy: 0.9554 - val_loss: 0.6133 - val_accuracy: 0.8067
Epoch 79/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1402 - accuracy: 0.9531 - val_loss: 0

110/110 [==============================] - 18s 165ms/step - loss: 0.1009 - accuracy: 0.9709 - val_loss: 0.4433 - val_accuracy: 0.8667
Epoch 129/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1018 - accuracy: 0.9691 - val_loss: 0.7067 - val_accuracy: 0.7893
Epoch 130/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1003 - accuracy: 0.9703 - val_loss: 0.7646 - val_accuracy: 0.7707
Epoch 131/200
110/110 [==============================] - 19s 168ms/step - loss: 0.1115 - accuracy: 0.9651 - val_loss: 0.5567 - val_accuracy: 0.8120
Epoch 132/200
110/110 [==============================] - 18s 165ms/step - loss: 0.1054 - accuracy: 0.9731 - val_loss: 0.5636 - val_accuracy: 0.8253
Epoch 133/200
110/110 [==============================] - 18s 164ms/step - loss: 0.1008 - accuracy: 0.9669 - val_loss: 0.5054 - val_accuracy: 0.8253
Epoch 134/200
110/110 [==============================] - 18s 164ms/step - loss: 0.0932 - accuracy: 0.9720 - val_loss: 0.6161 -

110/110 [==============================] - 18s 164ms/step - loss: 0.0879 - accuracy: 0.9720 - val_loss: 0.3768 - val_accuracy: 0.8773
Epoch 184/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0793 - accuracy: 0.9749 - val_loss: 0.4333 - val_accuracy: 0.8707
Epoch 185/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0797 - accuracy: 0.9726 - val_loss: 0.4281 - val_accuracy: 0.8680
Epoch 186/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0931 - accuracy: 0.9754 - val_loss: 0.4535 - val_accuracy: 0.8800
Epoch 187/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0879 - accuracy: 0.9754 - val_loss: 0.3156 - val_accuracy: 0.8893
Epoch 188/200
110/110 [==============================] - 18s 165ms/step - loss: 0.1130 - accuracy: 0.9703 - val_loss: 0.4363 - val_accuracy: 0.8627
Epoch 189/200
110/110 [==============================] - 18s 165ms/step - loss: 0.0907 - accuracy: 0.9749 - val_loss: 0.3585 -

In [ ]:
for